In [6]:
import numpy as np
import tensorflow as tf
import keras
import gym
import pygame
from keras import layers, optimizers, losses
import matplotlib.pyplot as plt
import time


In [7]:
class experience_memory():

    def __init__(self, buffer_capacity, batch_size, state_dim, action_dim):
        # Number of "experiences" to store at max
        self.buffer_capacity = buffer_capacity
        # Num of tuples to train on.
        self.batch_size = batch_size

        # Its tells us num of times record() was called.
        self.buffer_counter = 0

        # Instead of list of tuples as the exp.replay concept go
        # We use different np.arrays for each tuple element
        self.state_buffer = np.zeros((self.buffer_capacity, state_dim), dtype=np.float32)
        self.action_buffer = np.zeros((self.buffer_capacity, action_dim), dtype=np.int32)
        self.reward_buffer = np.zeros((self.buffer_capacity, 1), dtype=np.float32)
        self.next_state_buffer = np.zeros((self.buffer_capacity, state_dim), dtype=np.float32)
        self.done_buffer = np.zeros((self.buffer_capacity, 1), dtype=np.float32)

    # Takes (s,a,r,s') obervation tuple as input
    def record(self, obs_tuple):
        # Set index to zero if buffer_capacity is exceeded,
        # replacing old records
        index = self.buffer_counter % self.buffer_capacity

        self.state_buffer[index] = obs_tuple[0]
        self.action_buffer[index] = obs_tuple[1]
        self.reward_buffer[index] = obs_tuple[2]
        self.next_state_buffer[index] = obs_tuple[3]
        self.done_buffer[index] = obs_tuple[4]


        self.buffer_counter += 1

class MountainCar():

    def __init__(self):
        
        self.env = gym.make("CartPole-v1")

        self.batch_size = 32
        self.max_memory_size = 500000

        self.state_dim = 4
        self.action_dim = 1

        self.gamma = 0.99
        self.tau = 0.001
        self.lower_action_bound = -1
        self.upper_action_bound = 1

        self.action_space = np.array([0,1])
        self.num_a = len(self.action_space)

        self.buffer = experience_memory(self.max_memory_size, self.batch_size, self.state_dim, self.action_dim)

        # init the neural netsf
        self.critic = self.get_critic_NN()
        self.target_critic = self.get_critic_NN()
        self.target_critic.set_weights(self.critic.get_weights())
        self.eps = 1
        self.alpha = 0.001
        self.critic_optimizer = tf.keras.optimizers.Adam(self.alpha)

    @tf.function
    def update(self, state_batch, action_batch, reward_batch, next_state_batch, done_batch):
             
        
        next_q_vals =  self.target_critic(next_state_batch)
            
        target_vals = tf.reshape(tf.reduce_max(next_q_vals, axis =1),[self.batch_size,1])
        y = reward_batch + (1-done_batch)* self.gamma*target_vals
        y = tf.reshape(y,[self.batch_size,])
        mask = tf.reshape(tf.one_hot(action_batch, self.num_a, dtype=tf.float32),[self.batch_size, self.num_a])
        with tf.GradientTape() as tape:
            
            critic_value = self.critic(state_batch)
      
            critic_pred = tf.reduce_sum(tf.multiply(critic_value,mask), axis=1)
          
            #critic_loss = tf.reduce_mean(tf.math.square(y-critic_value))
            critic_loss = losses.MSE(y,critic_pred)
        
        
        critic_grad = tape.gradient(critic_loss, self.critic.trainable_variables)
        critic_grad, _ = tf.clip_by_global_norm(critic_grad, 2.0)
        self.critic_optimizer.apply_gradients(zip(critic_grad, self.critic.trainable_variables))

        
    def learn(self):
        # get sample

        record_range = min(self.buffer.buffer_counter, self.buffer.buffer_capacity)

        batch_indices = np.random.choice(record_range, self.batch_size)
        
        state_batch = tf.convert_to_tensor(self.buffer.state_buffer[batch_indices])
        action_batch = tf.convert_to_tensor(self.buffer.action_buffer[batch_indices])
        reward_batch = tf.convert_to_tensor(self.buffer.reward_buffer[batch_indices])
        next_state_batch = tf.convert_to_tensor(self.buffer.next_state_buffer[batch_indices])
        done_batch = tf.convert_to_tensor(self.buffer.done_buffer[batch_indices])
        
        self.update(state_batch, action_batch, reward_batch, next_state_batch,done_batch)

    @tf.function
    def update_target(self, target_weights, weights):
        for (a,b) in zip(target_weights, weights):
            a.assign(self.tau *b + (1-self.tau) *a)

    def get_critic_NN(self):
        # input [state, action]
        
        state_input = layers.Input(shape =(self.state_dim,))

        out = layers.Dense(32, activation = 'relu')(state_input)
        out = layers.BatchNormalization()(out)
        out = layers.Dense(32, activation = 'relu')(out)
        out = layers.Dense(self.num_a)(out)

        
        model = keras.Model(inputs = state_input, outputs = out)
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate= 0.001),
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.MeanSquaredError()],
        )
        return model

    def epsilon_greedy(self, state, eps):

        q_vals = self.critic(state)
        
        if (eps > np.random.rand()):
           
            rand_ind = np.random.choice(self.num_a)
            
            return self.action_space[rand_ind]
        
        else:
            
            a_ind = tf.argmax(q_vals,axis = 1)[0]
            
            return self.action_space[a_ind]
    
    def run_MC(self):
        done = False
        state = self.env.reset()
        state = tf.expand_dims(tf.convert_to_tensor(state),0)
        t_counter = 0
        while (True):
            self.env.render()
            time.sleep(0.01)
            
            a_ind = np.argmax(self.critic(state))
            
            action = self.action_space[a_ind]
            new_state, reward, done, info = self.env.step(action)
        
            new_state = tf.expand_dims(tf.convert_to_tensor(new_state.reshape(self.state_dim)),0)
            state = new_state
            t_counter += 1

            if (done):
                break
        print('Zeit: ', t_counter)
        

In [3]:
MC = MountainCar()

ep_reward_list = []
# To store average reward history of last few episodes
avg_reward_list = []
num_episode = 5000
decay = 0.9999
for ep in range(num_episode):
    
    done = False
    state = MC.env.reset()
    #state = tf.expand_dims(tf.convert_to_tensor(state),0)
    
    state = np.reshape(state, [1,MC.state_dim])
    episodic_reward = 0
    t_counter = 0

    #if (ep % 100 == 0 and ep != 0):
        #MC.run_MC()
    while(t_counter < 200):
        MC.alpha = 0.0001  #np.max([0.0001,0.01 * (1 / (1 +ep))])
        MC.eps = np.max([0.01, MC.eps * decay])
        action = MC.epsilon_greedy(state, MC.eps)
        
        new_state, reward, done, info = MC.env.step(action)
        
        #new_state = tf.expand_dims(tf.convert_to_tensor(new_state.reshape(MC.state_dim)),0)
       
        new_state = np.reshape(new_state, [1,MC.state_dim])
        episodic_reward += reward
        
        MC.buffer.record((state,action,reward, new_state, done))
       
        MC.learn()
    
        MC.update_target(MC.target_critic.variables, MC.critic.variables)

        state = new_state
        t_counter +=1
        if (done):
            break
    ep_reward_list.append(episodic_reward)
    # Mean of last 40 episodes
    avg_reward = np.mean(ep_reward_list[-20:])
    print("Episode * {} * AVG Reward is ==> {}, eps ==> {}".format(ep, avg_reward, MC.eps))
    avg_reward_list.append(avg_reward)
# Plotting graph
# Episodes versus Avg. Rewards
plt.plot(avg_reward_list)
plt.xlabel("Episode")
plt.ylabel("Avg. Epsiodic Reward")
plt.show()



(array([ 0.03865113, -0.0289197 ,  0.0269503 , -0.02833511], dtype=float32), {})


d:\Users\pillieri\Desktop\Visual Studio Code Projekte\Python Projekte\.venv\lib\site-packages\numpy\core\fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = getattr(asarray(obj), method)(*args, **kwds)


ValueError: cannot reshape array of size 2 into shape (1,4)